<a href="https://colab.research.google.com/github/NHirt32/LLM-News-Digest/blob/main/API_LLM_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup

Setting up the environment with required imports and installations


In [27]:
!pip install python-dotenv
!pip install openai

In [28]:
# For API requests
import requests
# For mounting files and securing sensitive data
from google.colab import drive
import os
from dotenv import load_dotenv
# For OpenAI requests
import openai

Here I'm mounting a .env file to prevent sharing my API key on my GitHub repository.

In [29]:
drive.mount('/content/drive')
env_path = '/content/drive/MyDrive/API-LLM-Project/.env'
load_dotenv(env_path)
news_key = os.getenv("NEWS_API_KEY")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# News API Setup

Setting up API parameters

In [30]:
# Top URLs syntax: url?country={chosen_country}&apiKey=API_KEY
top_headlines_url = 'https://newsapi.org/v2/top-headlines'
temp_country = 'US'
news_params = {
    'country': temp_country,
    'apiKey': news_key
}

Here, we retrieve a response from the News API. If an error occurs, we halt the execution of the notebook; otherwise, we combine the titles and descriptions into a single, readable string to provide to our LLM.

In [31]:
news_response = requests.get(top_headlines_url, params = news_params)
# If the response from News API is not "OK" stop the execution of the project
if news_response.status_code != 200:
  raise RuntimeError(f"Error: {news_response.status_code}, {news_response.reason}")
# Converting request object to dictionary
news_data = news_response.json()
# Now we combine all titles and descriptions into a single legible string
combined_content = '\n\n'.join(
    f"Title: {article['title']} \n Description{article['description']}"
    for article in news_data['articles']
)

# OpenAI API Setup

Here we setup our OpenAI API parameters.

In [35]:
openai.api_key = os.getenv("OPENAI_API_KEY")
openai_params = {
    'model': 'gpt-3.5-turbo',
    'messages': [
        {"role": "system", "content": "You are a helpful assistant that summarizes news articles."},
        {"role": "user", "content": f"{combined_content}"}

    ],
    # Response length variable
    'max_tokens': 100
}

Now using our API parameters we make a request to OpenAI to summarize the combined content of our News API data.

In [34]:
try:
  openai_response = await openai.chat.completions.create(**openai_params)
  article_summary = openai_response.choices[0].message['content'].strip()
  print(f"Summary of Articles: {article_summary}")
except Exception as e:
  print(f"An error occured: {e}")

An error occured: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
